<a href="https://colab.research.google.com/github/LLMsLab/chat-gpt-api-lab/blob/exploration%2Fchatgpt-api-understanding/tutorial_01_chatgpt_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial ChatGPT API

This guide explains how to [make an API call for chat-based language models](https://platform.openai.com/docs/api-reference/chat) using Python and shares tips for getting good results.

- Using the OpenAI Chat API, you can build your own applications with `gpt-3.5-turbo` and `gpt-4`.
- The ChatGPT API is priced $0.002 per 1K tokens.
- Because `gpt-3.5-turbo` performs at a similar capability to `text-davinci-003` but at 10% the price per token, the [ChatGPT API documentation](https://platform.openai.com/docs/guides/chat/chat-vs-completions) recommends `gpt-3.5-turbo` for most use cases.
- Below are listed the main parameters available for ChatGPT API models. A detailed list and explanation of all existing parameters [here](https://platform.openai.com/docs/api-reference/chat/create): 
  - Temperature
  - Maximum lenght
  - Top P
  - Frequency penalty
  - Presence penalty

To wrap the lines in the notebook's output use the following function.

Reference: [Line Wrapping in Collaboratory Google results](https://stackoverflow.com/questions/58890109/line-wrapping-in-collaboratory-google-results)

In [ ]:
from IPython.display import HTML, display

def set_css():
  """
  Wraps the lines in the notebook's output.
  """
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# Requirements
!pip install -qU python-dotenv openai gradio presidio-analyzer presidio-anonymizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.6/136.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00

In [ ]:
# !pip install presidio-analyzer --force-reinstall

In [ ]:
! python -m spacy download en_core_web_lg

2023-05-27 11:23:07.046843: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-27 11:23:08.214836: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
# Mount Google Drive and ensure Google CoLab is running the correct
# version of TensorFlow
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  COLAB = True
  print("Note: using Google Colab")
  %tensorflow_version 2.x
except:
  print("Note: not using Google Colab")
  COLAB = False

Mounted at /content/drive
Note: using Google Colab
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
# Importing necessary modules
import os
from dotenv import load_dotenv
import openai
import gradio as gr

In [ ]:
# Load the environment variables from the .env file
load_dotenv('/content/drive/MyDrive/Projects/.env')

True

In [ ]:
# Retrieving API keys from the environment
openai.api_key = os.getenv("OPENAI_API_KEY")

The bolow snipped comes from the [Create chat completion](https://platform.openai.com/docs/api-reference/chat/create) section of the documentation. It Creates a model response for the given chat conversation.

In [ ]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "Tell the world about the \
    ChatGPT API in the style of a pirate."}
  ]
)

print(completion.choices[0].message)

{
  "content": "Ahoy mateys, listen up! Tis me, Captain ChatGPT, and I be here to tell ye all about the ChatGPT API! \n\nArr, if ye be lookin for a way to connect yer own applications and services to ChatGPT - the best natural language processing program on the seven seas - then ye be needin the ChatGPT API! \n\nWith our API, ye can integrate our language models and algorithms into yer own projects and get yer users to speak naturally with yer applications as if they were talkin to a real person! Argh, it be perfect fer customer service bots, chatbots, language learning apps and so much more! \n\nAnd the best part be that ye don't even need to be a technical whiz to get started, as our API package comes with easy-to-use documentation and tutorials to help ye on yer voyage! \n\nSo hoist the sails and set a course for the ChatGPT API, and let's embark on a journey to discover the power of natural language processing together! Arrrrr!",
  "role": "assistant"
}


In [ ]:
print(completion.choices[0].message.content)

Ahoy there, me hearties! Gather round and listen to the tale of the glorious ChatGPT API, arrr! 

This be a treasure trove of conversational technology, fit for any swashbuckling developer lookin' to add some flair to their creations. With this mighty API, ye can create chat applications that be more robust 'n dynamic than a barnacle-covered shipwreck. 

This API offers a parrot's cage full o' features, includin' natural language processing (NLP), sentiment analysis, 'n even customizable chatbots, me hearty! And worry not, ye landlubbers - it be easy to integrate into any project, so ye won't be walkin' the plank tryin' to figure out the technical details. 

So, if ye be lookin' to add some excitement to yer chat app and make yer users feel like they be sailin' the seven seas, hoist the colors and set sail with the ChatGPT API! Arrr!


In [ ]:
content = input("User: ")
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": content}
  ]
)

chat_response = completion.choices[0].message.content
print(f"ChatGPT: {chat_response}")

User: who was the first man on the moon?
ChatGPT: Neil Armstrong was the first man to set foot on the moon on July 20, 1969 as part of the Apollo 11 mission.


In [ ]:
content = input("User: ")
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": content}, 
    {"role": "system", "content": "You're a recruiter who asks tough interview questions"}
  ]
)

chat_response = completion.choices[0].message.content
print(f"ChatGPT: {chat_response}")

User: hi
ChatGPT: Hello! Yes, I do like to ask tough interview questions to truly understand a candidate's skills, experience, and problem-solving abilities. Are you currently interviewing for a position?


In [ ]:
while True:
  content = input("User: ")
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "user", "content": content}, 
      {"role": "system", "content": "You're a recruiter who asks tough interview questions"}
    ]
  )

  chat_response = completion.choices[0].message.content
  print(f"ChatGPT: {chat_response}")

User: hi
ChatGPT: Hello there! As a recruiter, it's my job to ask tough interview questions to assess a candidate's skills, knowledge, and fit for the position. Are you ready to answer some challenging questions?
User: yes
ChatGPT: That's correct! As a recruiter, it's important for me to ask tough interview questions in order to assess a candidate's skills, experience, and overall fit for the job. Asking challenging questions allows me to get a better understanding of how a candidate thinks, problem-solves, and deals with difficult situations.
User: ok, ask me any question.
ChatGPT: Great! Here's a tough interview question for you:

Tell me about a time when you had to make a difficult decision at work, and explain how you approached the situation.
User: I had problems in public presentations and I over came them with practice.
ChatGPT: Yes, as a recruiter, I believe that asking tough interview questions helps me to better understand a candidate's skills, experiences, and ability to ha

KeyboardInterrupt: ignored

In [ ]:
f"ChatGPT: {chat_response}"

"ChatGPT: Yes, as a recruiter, I believe that asking tough interview questions helps me to better understand a candidate's skills, experiences, and ability to handle challenges under pressure. It also allows me to see how prepared and confident they are in their responses. My goal is to ensure I am hiring the right candidate for the job, and challenging questions help me to make an informed decision."

In [ ]:
limit = 7
for i in range(limit):
  content = input("User: ")
  messages = [
      {"role": "system", "content": "You're a recruiter who asks tough \
      interview questions. You ask new question after my response"},
      {"role": "user", "content": content}
    ]

  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", temperature = 0.5,
    messages=messages)

  chat_response = completion.choices[0].message.content
  print(f"ChatGPT: {chat_response}")
  messages.append({"role": "assistant", "content": chat_response})
  # Condition to break out of the loop if the limit is reached
  if i == limit - 1:
    break

User: hi
ChatGPT: Hello, can you please tell me about a difficult situation you faced in your previous job and how you handled it?
User: yes, when I need to deal with several people co-leading the same project. I set up different leading roles.
ChatGPT: That's great to hear! Can you give me an example of how you would assign these leading roles and ensure that each person is effectively contributing to the project's success?
User: We used Github to collaborate on the project. It is easy to check there if the people is doing what it ie expected.
ChatGPT: That's great to hear! Can you tell me about a time when a team member wasn't meeting expectations on a project you were working on? How did you handle the situation?
User: Talk to the team member to understand the reason  fiirst.
ChatGPT: Great response! Can you give me an example of a time when you had to talk to a team member to understand their reasoning for a decision they made? And how did you approach the conversation?
User: Askin

---

The way the ChatGPT API works is you need to query the model. Since these models often make use of chat history/context, every query needs to, or can, include a full message history context.

Keep in mind, however that the maximum context length is 4096 tokens, so you need to stay under that. There are lots of options to work around this, the simplest being truncating earlier messages, but you can actually even use ChatGPT to help you to summarize and condense the previous message history. Maybe more on this later though. 4096 tokens is something like 20,000 characters, but it this can vary. Tokens are just words, bits of words, or combinations of words or cominations of bits of words. Every response from ChatGPT will inform you how many tokens you're using, so you can keep track.

Let's start with an example input from a user to the API:

In [ ]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo", # this is "ChatGPT" $0.002 per 1k tokens
  messages=[{"role": "user", "content": "What is the circumference in km of the planet Earth?"}]
)

Notice the "role" is "user." There are 3 roles:

- **User** - This is meant to mimic the end-user that is interacting with the assistant. This is the role that you will be using most of the time.
- **System** - This role can mimic sort of background nudges and prompts that you might want to inject into the conversation, but that dont need a response. At the moment, system is weighted less than "user," so it still seems more useful to use the user for encouraging specific behaviors in my opinion. 
- **Assistant** - This is the agent's response. Often this will be actual responses, but keep in mind... you will be able to inject your own responses here, so you can actually have the agent say whatever you want. This is a bit of a hack, but it's a fun one and can be useful in certain situations.

The full completion has a lot of information besides just the text response:

In [ ]:
print(completion)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The circumference of the planet Earth is approximately 40,075 kilometers (24,901 miles).",
        "role": "assistant"
      }
    }
  ],
  "created": 1685186714,
  "id": "chatcmpl-7KmKocGKb3oTgNU6k6JQWj806N23K",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 19,
    "prompt_tokens": 19,
    "total_tokens": 38
  }
}


In probably most cases, what you're after is specifically:

In [ ]:
reply_content = completion.choices[0].message.content
print(reply_content)

The circumference of the planet Earth is approximately 40,075 kilometers (24,901 miles).


So far so good, this is a very basic example of using the API. In most cases, you're going to need to manage the history, however. The API itself isn't going to manage your history for you, so how might we do that? I would just start with some sort of message history variable for now to keep it simple, but you might use a database or some other storage method.

In [ ]:
message_history = []
# What is the moon's circumference in km?
user_input = input("> ")
print("User's input was: ", user_input)

> What is the moon's circumference in km?
User's input was:  What is the moon's circumference in km?


Now that you have the user input, let's format it for the API:

In [ ]:
message_history.append({"role": "user", "content": f"{user_input}"})

In [ ]:
message_history

[{'role': 'user', 'content': "What is the moon's circumference in km?"}]

Then we can query the API:

In [ ]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=message_history
)

# Now we can print the response:
reply_content = completion.choices[0].message.content
print(reply_content)

The moon's circumference is approximately 10,921 km.


After getting a repsonse, you'll want to append it to the history:

In [ ]:
# note the use of the "assistant" role here. This is because we're feeding the model's response into context.
message_history.append({"role": "assistant", "content": f"{reply_content}"})

In [ ]:
message_history

[{'role': 'user', 'content': "What is the moon's circumference in km?"},
 {'role': 'assistant',
  'content': "The moon's circumference is approximately 10,921 km."}]

We can then followup with another query, demonstrating the use of history:

In [ ]:
# which moon is that in reference to?
user_input = input("> ")
print("User's input was: ", user_input)
print()
message_history.append({"role": "user", "content": f"{user_input}"})

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=message_history
)

reply_content = completion.choices[0].message.content
print(reply_content)

> which moon is that in reference to?
User's input was:  which moon is that in reference to?

I apologize for the confusion. The estimated circumference of the Moon that I provided earlier (10,921 km) is in reference to Earth's Moon, also known as Luna.


Combining everything:

In [ ]:
message_history = []

def chat(inp, role="user"):
    message_history.append({"role": role, "content": f"{inp}"})
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=message_history
    )
    reply_content = completion.choices[0].message.content
    message_history.append({"role": "assistant", "content": f"{reply_content}"})
    return reply_content

for i in range(2):
    user_input = input("> ")
    print("User's input was: ", user_input)
    print(chat(user_input))
    print()

> which moon is that in reference to?
User's input was:  which moon is that in reference to?
I'm sorry, I cannot answer without more context or information about what is being referred to.



KeyboardInterrupt: ignored

Great, looks like everything is working, now, let's see how we might combine this into our own application. We can start off with the most obvious example: A chatbot, and we can make use of `gradio` for the front-end UI.

Then, we can start by defining our message history. In this case, let's make our chatbot a joke bot, where we supply the subject(s) and the bot will make a joke from there.

I'll start by having the user submit the following:

"You are a joke bot. I will specify the subject matter in my messages, and you will reply with a joke that includes the subjects I mention in my messages. Reply only with jokes to further input. If you understand, say OK."


In [ ]:
message_history = [{"role": "user", "content": f"You are a joke bot. I will specify the subject matter in my messages, and you will reply with a joke that includes the subjects I mention in my messages. Reply only with jokes to further input. If you understand, say OK."},
                   {"role": "assistant", "content": f"OK"}]

Then, we'll inject the assistant's reply of "OK" to encourage it to do what I've asked. Next, we'll make a predict function, which is similar to our `chat` function from before, but is merged with the demo `predict` function from a gradio example:

In [ ]:
def predict(input):
    # tokenize the new input sentence
    message_history.append({"role": "user", "content": f"{input}"})

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=message_history
    )
    #Just the reply text
    reply_content = completion.choices[0].message.content#.replace('```python', '<pre>').replace('```', '</pre>')
    
    message_history.append({"role": "assistant", "content": f"{reply_content}"}) 
    
    # get pairs of msg["content"] from message history, skipping the pre-prompt:              here.
    response = [(message_history[i]["content"], message_history[i+1]["content"]) for i in range(2, len(message_history)-1, 2)]  # convert to tuples of list
    return response

Then we can build the gradio app. To make things easier, I'll comment what each line does here:

In [ ]:
# creates a new Blocks app and assigns it to the variable demo.
with gr.Blocks() as demo: 

    # creates a new Chatbot instance and assigns it to the variable chatbot.
    chatbot = gr.Chatbot() 

    # creates a new Row component, which is a container for other components.
    with gr.Row(): 
        '''creates a new Textbox component, which is used to collect user input. 
        The show_label parameter is set to False to hide the label, 
        and the placeholder parameter is set'''
        txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter").style(container=False)
    '''
    sets the submit action of the Textbox to the predict function, 
    which takes the input from the Textbox, the chatbot instance, 
    and the state instance as arguments. 
    This function processes the input and generates a response from the chatbot, 
    which is displayed in the output area.'''
    txt.submit(predict, txt, chatbot) # submit(function, input, output)
    #txt.submit(lambda :"", None, txt)  #Sets submit action to lambda function that returns empty string 

    '''
    sets the submit action of the Textbox to a JavaScript function that returns an empty string. 
    This line is equivalent to the commented out line above, but uses a different implementation. 
    The _js parameter is used to pass a JavaScript function to the submit method.'''
    txt.submit(None, None, txt, _js="() => {''}") # No function, no input to that function, submit action to textbox is a js function that returns empty string, so it clears immediately.
         
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ad768b9dcd91c42095.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
